In [4]:
import os 
import utils.visulaiser as visulaiser
from datasets import load_dataset, load_from_disk

from torch import nn
from tqdm import tqdm
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt
from datasets import load_dataset
import re
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.optim import AdamW
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
print(device)

cuda


# Model

In [6]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
base_model = AutoModelForCausalLM.from_pretrained(model_name)

2025-03-12 00:36:48.657067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-12 00:36:48.657134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-12 00:36:48.659119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 00:36:48.671017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class ModifiedModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.qwen = base_model.model
        self.linear = nn.Linear(896, 1)

    def forward(self, inputs):
        outputs = self.qwen(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]
        return self.linear(cls_token)

# Data

In [12]:
# Load GSM8K dataset
# dataset = load_dataset("gsm8k", "main")
dataset = load_from_disk("./gsm8k_saved")

def extract_final_answer(answer):
    """
    Extracts only the numerical value after '####' in the answer field.
    """
    match = re.search(r"####\s*([\d\.]+)", answer)  # Match number after ####
    return float(match.group(1)) if match else 0  # Return extracted number

# Process training and test sets
for split in ["train", "test"]:
    dataset[split] = dataset[split].map(lambda example: {
        "question": example["question"],
        # "answer": tokenizer(extract_final_answer(example["answer"]),
        #                     padding='max_length',
        #                     truncation=True,
        #                     max_length=16,
        #                     return_tensors='pt').to(device),
        "answer": extract_final_answer(example["answer"])
    })

# Save processed dataset
dataset.save_to_disk("./gsm8k_cleaned")

# Print an example to verify
print(dataset["train"][0])

# Split into train and test sets
train_data = dataset["train"]
test_data = dataset["test"]

Saving the dataset (0/1 shards):   0%|          | 0/7473 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 72.0}


In [13]:
def format_example(example):
    # print(example)
    return f"Question: YOU ARE A EXPERT AT MATH. NOW ANSWER THIS QUESTION - {example['question']}. REPLY JUST THE FINAL ANSWER AS A NUMBER. Answer: "

# Tokenize data
def preprocess_function(examples):
    texts = format_example(examples)
    tokens = tokenizer(texts, 
                     padding="max_length", 
                     truncation=True, 
                     max_length=128, 
                     return_tensors="pt")
    return tokens

# Apply preprocessing
tokenized_train = train_data.map(preprocess_function, batched=False)
tokenized_test = test_data.map(preprocess_function, batched=False)

# Rename 
tokenized_train = tokenized_train.remove_columns('question')
# tokenized_train = tokenized_train.rename_column('answer', 'labels')

tokenized_test = tokenized_test.remove_columns('question')
# tokenized_test = tokenized_test.rename_column('answer', 'labels')

In [14]:
small_train_dataset = tokenized_train.shuffle(seed=42)#.select(range(1000))
small_eval_dataset = tokenized_test.shuffle(seed=42)

In [15]:


train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=1)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=1)

# PyTorch Bare Bones

In [16]:
def train(model, dataloader, loss_metric, optimizer, num_epochs):
    num_training_steps = num_epochs * len(dataloader)
    progress_bar = tqdm(range(num_training_steps))
    
    model = ModifiedModel(base_model)
    model = model.to(device)
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=5e-3)
    loss_metric = nn.MSELoss()
    
    loss_arr = []
    
    for epoch in range(num_epochs):
        running_loss = 0
        for i, sample in enumerate(dataloader):
            batch = {k: torch.tensor(v).to(device) for k, v in sample.items() if k != 'answer'}
            
            output = model(batch)
            if isinstance(output, tuple):  # Ensure proper indexing
                output = output[0]
            
            # Ensure shape consistency for loss calculation
            loss = loss_metric(output.view(-1, 1).float(), sample['answer'].view(-1, 1).to(device).float())
    
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            progress_bar.update(1)
    
            running_loss += loss.item()
            if (i % 100 == 0):
                print(output.item(), sample['answer'].item())
                print(f"Step {i}: Loss = {loss.item()}")
    
        loss_arr.append(running_loss / len(train_dataloader))
    
    print("Training complete!")
    return loss_arr
    

In [17]:
model = ModifiedModel(base_model)

optimizer = AdamW(model.parameters(), lr=5e-3)
loss_metric = nn.MSELoss()

In [9]:
train(model, train_dataloader, loss_metric, optimizer, num_epochs=5)

NameError: name 'train' is not defined

In [36]:
# print(torch.cuda.memory_summary(device=None, abbreviated=True))

In [18]:
# Empty the GPU cache
torch.cuda.empty_cache()

# Reset the peak memory stats
torch.cuda.reset_peak_memory_stats()

In [ ]:
# Save the model
torch.save(model.state_dict(), './qwen_gsm8k_finetuned/pytorch_model.bin')
tokenizer.save_pretrained("./qwen_gsm8k_finetuned")

In [ ]:
# Load the model again
model = ModifiedModel(base_model)  # Recreate your modified model
model.load_state_dict(torch.load('./qwen_gsm8k_finetuned/pytorch_model.bin'))

# Evaluate

In [25]:
def evaluate(model, dataloader, device):
    model = model.to(device)
    model.eval()

    num_training_steps = len(dataloader)
    progress_bar = tqdm(range(num_training_steps))

    running_loss = 0
    nCorrect = 0

    with torch.no_grad():
        for i, sample in enumerate(dataloader):
            batch = {k: torch.tensor(v).to(device) for k, v in sample.items() if k != 'answer'}
            
            output = model(batch)
            if isinstance(output, tuple):  # Ensure proper indexing
                output = output[0]
            
            # Ensure shape consistency for loss calculation
            loss = loss_metric(output.view(-1, 1).float(), sample['answer'].view(-1, 1).to(device).float())
            nCorrect += (output.view(-1, 1).float() == sample['answer'].view(-1, 1).to(device).float()) 

            optimizer.step()
            optimizer.zero_grad()
            progress_bar.update(1)

            running_loss += loss.item()
            if (i % 100 == 0):
                print(f"Step {i}: Loss = {loss.item()}")

    return nCorrect / len(dataloader)

In [26]:
acc = evaluate(model, eval_dataloader, device)

  0%|          | 0/1319 [00:00<?, ?it/s]

Step 0: Loss = 12708.708984375
Step 100: Loss = 899.7828979492188
Step 200: Loss = 3.757678508758545
Step 300: Loss = 47496.15625
Step 400: Loss = 247942.71875
Step 500: Loss = 288.5423278808594
Step 600: Loss = 104895.5078125
Step 700: Loss = 625088448.0
Step 800: Loss = 128.3656768798828
Step 900: Loss = 2277.062744140625
Step 1000: Loss = 15.132217407226562
Step 1100: Loss = 658.2655029296875
Step 1200: Loss = 487.9842834472656
Step 1300: Loss = 668.4796142578125


In [27]:
print("Test Accuracy:" , acc)

Test Accuracy: tensor([[0.]], device='cuda:0')
